In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tomllib
import shapely

In [ ]:
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
import read_iso
import arya

In [ ]:
obj = "yasone1"
julen = False

In [ ]:
r_h = 25 / 60

In [ ]:
r_inner = 40  / 60
r_ring = 2*r_inner

# Data loading

In [ ]:
def get_mag_shift(objname):
    shifts = {}
    with open("../photometry/" + objname + "/panstarrs_shift.toml", "rb") as f:
        shifts = tomllib.load(f)
    return shifts

In [ ]:
def read_catalogue(objname):
    cat = Table.read(f"../photometry/{objname}/allcolours.cat")

    shifts = get_mag_shift(objname)
    cat["GMAG"] -= shifts["g"]
    cat["RMAG"] -= shifts["r"]
    cat["IMAG"] -= shifts["i"]
    cat = cat[(cat["RFLAGS"] < 8 ) & (cat["RFLAGS_WEIGHT"] == 0) ]
    return cat

In [ ]:
def read_catalogue_julen(objname):
    cat = Table.read(f"{objname}_julen.cat")

    return cat

In [ ]:
get_mag_shift(obj)

In [ ]:

if julen:
    cat = read_catalogue_julen(obj)
else:
    cat = read_catalogue(obj)
    cat = cat[cat["RFLAGS"] & 4 == 0]


# Analysis

In [ ]:
i = 0
j = 1
k = 3
l = 1


x = cat["MAG_APER"][:, i] - cat["MAG_APER"][:, j]
y = -cat["MAG_APER"][:, k] + cat["MAG_APER"][:, l]
flux_param = y - 0.5*x

In [ ]:
μ_flux, med_flux, std_flux = sigma_clipped_stats(flux_param[(cat["FLUX_RADIUS"] < 3)
    & (cat["RMAG"] < 21) & (cat["ELLIPTICITY"] < 0.3)
    ], stdfunc="mad_std", sigma=3)

In [ ]:
flux_thresh = med_flux + 3*std_flux

In [ ]:
fig, axs = plt.subplots(4, 1, sharex=True, gridspec_kw={"hspace": 0}, figsize=(6, 6))

plt.sca(axs[0])
plt.scatter(flux_param, np.log10(cat["FLUX_RADIUS"]), s=0.3, lw=0)
plt.ylabel("log flux radius")
plt.axvline(flux_thresh)


plt.sca(axs[1])

plt.scatter(flux_param, np.log10(cat["KRON_RADIUS"]), s=0.3, lw=0)
plt.ylabel("log kron radius")
plt.axvline(flux_thresh)

plt.sca(axs[2])

plt.scatter(flux_param, np.log10(cat["FWHM_WORLD"].to("arcsec")/u.arcsec), s=0.3, lw=0)
plt.ylabel("log fwhm/arcsec")
plt.axvline(flux_thresh)


plt.sca(axs[3])

plt.scatter(flux_param, 3*(cat["ELLIPTICITY"]), s=0.1, lw=0)
plt.axvline(flux_thresh)
plt.ylabel("ell")

plt.xlim(-2, 1)
plt.xlabel("flux param")

In [ ]:


c = plt.scatter(cat["RMAG"],  flux_param, s=0.3, alpha=1, c=np.log10(cat["FLUX_RADIUS"]), lw=0, label="all sources", vmax=0.5)

plt.colorbar(c)
plt.axhline(flux_thresh)

plt.ylim(-1.1, 0.5)
plt.ylim(-2, 2)
# plt.xlim(1.8, 0.7)
plt.xlabel(f"I{i} - I{j}")
plt.ylabel(f"I{l} - I{k}")

In [ ]:
from astropy.stats import sigma_clipped_stats

In [ ]:
c = plt.scatter(x,  flux_param, s=0.3, alpha=1, c=np.log10(cat["FLUX_RADIUS"]), lw=0, label="all sources", vmax=1)

plt.colorbar(c)
# plt.axhline(0.1)
# plt.axhline(-0.3)
plt.axhline(flux_thresh)

plt.ylim(-1.1, 0.5)
plt.ylim(-2, 2)
plt.xlim(1.8, 0.7)

plt.xlabel(f"I{i} - I{j}")
plt.ylabel(f"I{l} - I{k}")

In [ ]:

tab = cat
plt.scatter(tab["FLUX_RADIUS"] , -tab["MAG_APER"][:, k] + tab["MAG_APER"][:, l], s=0.3, alpha=1, c="black", lw=0)
plt.ylim(-5)
plt.axhline(0.35)
plt.xlim(0, 10)

In [ ]:
p = plt.scatter(cat["GMAG"] - cat["RMAG"], cat["RMAG"] - cat["IMAG"], s=0.3, c=flux_param, vmin=-0.2, vmax=0.5)
plt.xlabel("g - r")
plt.ylabel("r - i")
plt.xlim(-1, 2)
plt.ylim(-0.5, 2)
plt.colorbar(label="flux param")

In [ ]:
p = plt.scatter(cat["GMAG"] - cat["RMAG"], cat["RMAG"] - cat["IMAG"], s=0.3, c=np.log10(cat["FLUX_RADIUS"]), vmax=0.6)
plt.xlabel("g - r")
plt.ylabel("r - i")
plt.xlim(-1, 2)
plt.ylim(-0.5, 2)
plt.colorbar(label="log flux radius")